In [1]:
import numpy as np
#import jax.numpy as jnp

data = np.loadtxt("/Papilonidae_dataset_v2/Papilionidae_aligned_new.txt", delimiter="\t").reshape((2240, 200))


In [2]:
from ete3 import Tree

# Load the tree and get the leaf order in inorder traversal
ptree = Tree("/var/home/luka/proj/Papilonidae_dataset_v2/papilionidae_tree.txt", format=1)
order = ptree.get_leaf_names()  # returns in-order leaves (same as pre-order when just looking at leaves)


In [3]:
import pandas as pd

# Load categories
categories = pd.read_csv("/Papilonidae_dataset_v2/Papilonidae_metadata_new.txt", header=None)[0]

# Create DataFrame from data
df = pd.DataFrame(data.reshape(2240, -1))
df['category'] = pd.Categorical(categories, categories=categories.unique())

# Group by category and calculate means
means = df.groupby('category', observed=True).mean()

# Add order column based on the tree order
df['order'] = pd.Categorical(df['category'], categories=order, ordered=True)


# Reorder means DataFrame based on the order column
means = means.reset_index()
means['order'] = pd.Categorical(means['category'], categories=order, ordered=True)
means = means.sort_values('order')

# Drop unnecessary columns and convert to numpy array
X = means.drop(columns=['category', 'order']).values

N, d = X.shape

In [4]:
D = np.zeros((N*d, d))

# Create an array of indices
i_indices = np.arange(N*d)
j_indices = np.arange(d)

# Use broadcasting to create a mask
mask = (j_indices[:, None] * N <= i_indices) & (i_indices < (j_indices[:, None] + 1) * N)

D[mask.T] = 1.0


In [67]:
preorder = [n for n in ptree.traverse("preorder")]
M = len(preorder)   # number of nodes in tree (including internal)
dists = np.zeros((M))
inds = np.zeros(N, dtype=int)
inds_r = np.zeros(M, dtype=int)

j = 0
for i in range(M):
    n = preorder[i]
    dists[i:i+len(n.get_descendants())+1] += preorder[i].dist
    if n.name[0] != 'Q':
        inds[j] = i
        inds_r[i] = j
        j += 1


In [64]:

# Step 1: Euler Tour and Depth Array
def euler_tour(node):
    global euler, depth, first_occurrence, current_depth
    euler = []
    depth = []
    first_occurrence = {}
    current_depth = 0

    def euler_tour_rec(node):
        global euler, depth, first_occurrence, current_depth
        euler.append(node)
        depth.append(current_depth)
        if node not in first_occurrence:
            first_occurrence[node] = len(euler) - 1
            #print(first_occurrence)
        current_depth += 1
        for child in node.children:
            euler_tour_rec(child)
            euler.append(node)
            depth.append(current_depth)
        current_depth -= 1

    euler_tour_rec(node)
    #return euler, depth, first_occurrence

# Step 2: Build Segment Tree for RMQ
def build_segment_tree():
    global st, depth, n
    n = len(depth)
    st = [0] * (2 * n)
    for i in range(n):
        st[n + i] = i
    for i in range(n - 1, 0, -1):
        if depth[st[i * 2]] < depth[st[i * 2 + 1]]:
            st[i] = st[i * 2]
        else:
            st[i] = st[i * 2 + 1]
    #return st

def rmq(l, r):
    global st, depth, n
    res = l
    while l < r:
        if l % 2:
            if depth[st[l]] < depth[st[res]]:
                res = st[l]
            l += 1
        if r % 2:
            r -= 1
            if depth[st[r]] < depth[st[res]]:
                res = st[r]
        l //= 2
        r //= 2
    return res

# Step 3: LCA Query
def lca(u, v):
    global first_occurrence, euler, depth, n, st
    if first_occurrence[u] > first_occurrence[v]:
        u, v = v, u
    l = first_occurrence[u]
    r = first_occurrence[v]
    #print(f"l: {l}, r: {r}")  # Debugging statement
    index = rmq(l+n, r+n+1) - n
    #print(f"index: {index}")  # Debugging statement
    return euler[index]


In [104]:
global euler, depth, first_occurrence, current_depth
euler = depth = first_occurrence = current_depth = None

leaves = ptree.get_leaves()
#euler, depth, first_occurrence = euler_tour(ptree)
#st = build_segment_tree(depth)

euler_tour(ptree)
build_segment_tree()
#print(depth)

lca_matrix = np.zeros((N, N), dtype=int)
lca2r_dists = np.zeros((N, N))

for i in range(N):
    for j in range(i, N):
        #ancestor = lca(leaves[i], leaves[j])
        ancestor = leaves[i].get_common_ancestor(leaves[j])
        lca_ij = preorder.index(ancestor)

        #    print(f'{leaves[i].name:<35}{leaves[j].name:<35}{ancestor.name:<35}{dists[lca_ij]}')
        lca_matrix[i, j] = lca_matrix[j, i] = lca_ij
        lca2r_dists[i, j] = lca2r_dists[j, i] = dists[lca_ij]
        #if i == j:
        #    print(i, lca_ij, inds_r[i])


for i in range(N):
    print(lca2r_dists[i],lca2r_dists[i,i])
    j = np.argmax(lca2r_dists[i])
    print(f'{i:<10}{j:<10}{i == j}')

[0.1711 0.     0.     0.     0.     0.     0.     0.     0.     0.
 0.     0.     0.     0.     0.     0.     0.     0.     0.     0.
 0.     0.     0.     0.     0.     0.     0.     0.     0.     0.
 0.     0.     0.     0.     0.     0.     0.     0.     0.     0.
 0.     0.     0.     0.     0.     0.     0.     0.    ] 0.1711
0         0         True
[0.      0.19529 0.11057 0.11057 0.11057 0.11057 0.04527 0.04527 0.04527
 0.04527 0.04527 0.04527 0.04527 0.04527 0.04527 0.04527 0.04527 0.04527
 0.04527 0.04527 0.04527 0.04527 0.04527 0.04527 0.04527 0.04527 0.04527
 0.04527 0.04527 0.04527 0.04527 0.04527 0.04527 0.04527 0.04527 0.04527
 0.04527 0.04527 0.04527 0.04527 0.04527 0.04527 0.04527 0.04527 0.04527
 0.04527 0.04527 0.04527] 0.19529000000000002
1         1         True
[0.      0.11057 0.25351 0.18468 0.18468 0.12372 0.04527 0.04527 0.04527
 0.04527 0.04527 0.04527 0.04527 0.04527 0.04527 0.04527 0.04527 0.04527
 0.04527 0.04527 0.04527 0.04527 0.04527 0.04527 0.04527 0.0

In [99]:

def ecov(a,b):
    na,nb = order[a],order[b]
    if a == b:
        mrca = ptree.search_nodes(name=order[a])[0]
    else:
        mrca = ptree.get_common_ancestor(na,nb)

    ds = ptree.get_distance(mrca)
    #print(f'{na:>30}\t{nb:>30}\t{mrca.name:>30}\t{dist}\n')

    return ds

Cov = np.empty((N,N))

for i in range(N):
    for j in range(N):
        Cov[i,j] = ecov(i,j)

for i in range(N):
    for j in range(N):
        a = Cov[i,j]
        b = lca2r_dists[i,j]
        if a != b:
            print(a-b)

-2.7755575615628914e-17
-2.7755575615628914e-17
-2.7755575615628914e-17
-2.7755575615628914e-17
-5.551115123125783e-17
-2.7755575615628914e-17
-2.7755575615628914e-17
-2.7755575615628914e-17
-5.551115123125783e-17
-2.7755575615628914e-17
2.7755575615628914e-17
2.7755575615628914e-17
2.7755575615628914e-17
2.7755575615628914e-17
-2.7755575615628914e-17
-1.3877787807814457e-17
-1.3877787807814457e-17
-1.3877787807814457e-17
-2.7755575615628914e-17
-1.3877787807814457e-17
-1.3877787807814457e-17
-1.3877787807814457e-17
-1.3877787807814457e-17
-1.3877787807814457e-17
-2.7755575615628914e-17
-2.7755575615628914e-17
-1.3877787807814457e-17
-1.3877787807814457e-17
-2.7755575615628914e-17
2.7755575615628914e-17
-1.3877787807814457e-17
-1.3877787807814457e-17
-2.7755575615628914e-17
5.551115123125783e-17
2.7755575615628914e-17
-2.7755575615628914e-17
-2.7755575615628914e-17
-2.7755575615628914e-17
-2.7755575615628914e-17
-2.7755575615628914e-17
-2.7755575615628914e-17
-2.7755575615628914e-17
-2

In [60]:
ls = ptree.get_leaves()
#print(ls[0], type(ls))
i = 0
for n in ptree.traverse("preorder"):
    na = n.name
    m = ls[i]
    if na[0] != 'Q':
        print(na)
        #lna = ls[i]
        #print(na,'\t',rna,'\t',na == rna)
        #print(f'{na:_<30}{len(n.get_ancestors())}')#{lna:_<40}{na==lna}')
        i += 1


Baronia_brevicornis
Iphiclides_podalirius
Graphium_evemon
Graphium_sarpedon
Graphium_agamemnon
Protographium_marcellus
Hypermnestra_helios
Parnassius_orleans
Parnassius_honrathi
Archon_apollinus
Luehdorfia_puziloi
Sericinus_montela
Zerynthia_polyxena
Allancastria_cerisyi
Teinopalpus_imperialis
Battus_polydamas
Battus_belus
Pharmacophagus_antenor
Trogonoptera_brookiana
Troides_rhadamantus
Ornithoptera_richmondia
Euryades_corethrus
Parides_agavus
Parides_photinus
Parides_eurimedes
Cressida_cressida
Byasa_alcinous
Atrophaneura_semperi
Atrophaneura_dixoni
Pachliopta_aristolochiae
Pachliopta_kotzebuea
Losaria_coon
Papilio_aristodemus
Papilio_thoas
Papilio_cresphontes
Papilio_slateri
Papilio_glaucus
Papilio_troilus
Papilio_gigon
Papilio_xuthus
Papilio_polyxenes
Papilio_zelicaon
Papilio_deiphobus
Papilio_protenor
Papilio_polytes
Papilio_phestus
Papilio_ambrax
Meandrusa_sciron


In [91]:
N

48

In [61]:
v1 = jnp.ones(N)
evoCov_inv = jnp.linalg.inv(Cov)
tmp = v1.T @ evoCov_inv
mle_r = ((tmp @ v1) **-1) * (tmp @ X)
assert mle_r.shape==(d,)

tmp = X - mle_r.T
mle_R = (((N - 1) ** -1) * tmp.T) @ evoCov_inv @ tmp
assert mle_R.shape==(d,d)

In [62]:
X_mean = jnp.mean(X,axis=0)
evals, evecs = jnp.linalg.eigh(Cov)
X_cent = X - X_mean[None,:]

def ppca_recon(k=2):
    V_k = evecs[:, -k:]
    #print(X_cent.shape,V_k.shape)
    X_reduced = X_cent.T @ V_k
    #print(X_reduced.shape)
    X_reconstructed = X_reduced @ V_k.T + X_mean[:,None]
    #print(X_reconstructed.shape)
    return X_reconstructed

runtime modul

In [64]:
for k in range(1,N+1):
    ppca_recon(k)